In [ ]:
import sys

# TODO: Append the path to your `multi-annotator-machine-learning` project.
sys.path.append("../../")

from skactiveml.utils import ExtLabelEncoder
from maml.data import Dopanim
from maml.utils import (
    plot_annotation_calibration_curves,
    plot_annotation_times_histograms,
    plot_annotation_confusion_matrices,
    plot_annotator_metadata_correlation,
)

# TODO: Adjust data path to your dataset.
DATA_PATH = "."

# TODO: Adjust flag for downloading the dataset.
DOWNLOAD = False

# TODO: Adjust path for saving figures.
FIGURE_PATH = "."

### Load Data

In [ ]:
train_ds = Dopanim(DATA_PATH, version='train', variant='full', download=DOWNLOAD)
y_true, observation_ids = train_ds.load_true_class_labels()
le = ExtLabelEncoder(missing_label=None).fit(y_true)
y_true_enc = le.transform(y_true)
annotation_inconsistencies = train_ds.load_annotation_consistencies(
    observation_ids=observation_ids, annotators=train_ds.annotators
)
annotation_times = train_ds.load_annotation_times(
    observation_ids=observation_ids,
    annotators=train_ds.annotators,
)
likelihoods = train_ds.load_likelihoods(
    observation_ids=observation_ids,
    annotators=train_ds.annotators,
    classes=le.classes_,
    normalize=True,
)
annotator_metadata, annotator_feature_names = train_ds.load_annotator_metadata(
    annotators=train_ds.annotators
)

### Plot Calibration Curves

In [ ]:
plot_annotation_calibration_curves(
    likelihoods=likelihoods, y_true=y_true, classes=train_ds.classes, n_bins=10, path=FIGURE_PATH,
)

### Plot Annotation Times

In [ ]:
plot_annotation_times_histograms(times=annotation_times, n_bins=50, path=FIGURE_PATH)

### Plot Confusion Matrices

In [ ]:
cm_all = plot_annotation_confusion_matrices(
    likelihoods=likelihoods,
    classes=train_ds.classes,
    y_true=y_true,
    path=FIGURE_PATH,
)

### Plot Annotator Metadata

In [ ]:
print(annotator_feature_names)
sub_indices = [12, 13, 4, 3, 9, 0]
annotator_feature_names_sub = annotator_feature_names[sub_indices]
annotator_metadata_sub = annotator_metadata[:, sub_indices]
plot_annotator_metadata_correlation(
    annotator_metadata=annotator_metadata_sub,
    annotator_feature_names=annotator_feature_names_sub,
    cm_all=cm_all,
    path=FIGURE_PATH,
)

### Plot Annotation Time as a Function of Entropy

In [ ]:
likelihoods_new = likelihoods.reshape(-1, likelihoods.shape[-1])
annotation_times_new = annotation_times.ravel()
is_annotated = annotation_times_new != -1
likelihoods_new = likelihoods_new[is_annotated]
annotation_times_new = annotation_times_new[is_annotated]
entropies = entr(likelihoods_new).sum(axis=1)
bins = np.linspace(0, 1.75, num=10)
binned_entropies = np.digitize(entropies, bins)
fontsize = 18

# Calculate the average annotation time for each bin
average_times = np.array([np.mean(annotation_times_new[binned_entropies == i]) for i in range(1, len(bins))])
counts = np.array([np.sum(binned_entropies == i) for i in range(1, len(bins))])

# Calculate the center of each bin for plotting
bin_centers = (bins[:-1] + bins[1:]) / 2

# Plot the average annotation time per bin
fig = plt.figure(figsize=(7, 5), dpi=100)  # plt.subplots_adjust(0, 0, 1, 1)
ax_histogram = fig.add_axes([0.1, 0.1, 0.8, 0.8])  # Manually set the axes position
ax_time = ax_histogram.twinx()
ax_histogram.bar(bin_centers, counts, width=(bins[1] - bins[0]), color="#008080ff")
ax_time.plot(bin_centers, average_times, color="#800080ff", lw=3)
plt.savefig(FIGURE_PATH + "/times_entropy.pdf")
plt.show()